In [1]:
from Encoder import *
from Decoder import *

In [2]:
Decoder(20,20,10)

Decoder(
  (attention): Sequential(
    (0): Linear(in_features=40, out_features=10, bias=False)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=1, bias=False)
  )
  (lstm): LSTM(40, 20)
  (out): Linear(in_features=20, out_features=20, bias=True)
)

In [3]:
class ANMT(nn.Module):
    def __init__(self, height: int, width: int, input_channel: int, vocab_size: int, hidden_size: int, attention_size: int, num_layers=1, drop_prob=0) -> None:
        super(ANMT, self).__init__()
        self.Encoder = Encoder(height, width, input_channel, hidden_size)
        self.Decoder = Decoder(vocab_size, hidden_size, attention_size, num_layers, drop_prob)
    
    def forward(self, x: torch.Tensor):
        embbeding = self.Encoder(x)
        embbeding = embbeding.view(embbeding.shape[0], embbeding.shape[1], -1).permute(2,0,1).unsqueeze(0)
        state = embbeding[:,-1,:,:]

        for inp in embbeding:
            output, state = self.Decoder(inp, state, embbeding)

        return output

In [4]:
model = ANMT(7, 7, 2048, 20, 300, 10)

In [5]:
model

ANMT(
  (Encoder): Encoder(
    (model): MDLSTM(
      (lstm_lr_tb): MDLSTMCell()
      (lstm_rl_tb): MDLSTMCell()
      (lstm_lr_bt): MDLSTMCell()
      (lstm_rl_bt): MDLSTMCell()
      (fold): Fold(output_size=(7, 7), kernel_size=(1, 1), dilation=1, padding=0, stride=1)
    )
  )
  (Decoder): Decoder(
    (attention): Sequential(
      (0): Linear(in_features=600, out_features=10, bias=False)
      (1): Tanh()
      (2): Linear(in_features=10, out_features=1, bias=False)
    )
    (lstm): LSTM(600, 300)
    (out): Linear(in_features=300, out_features=20, bias=True)
  )
)

In [6]:
img = torch.zeros(1, 2048, 7, 7)
img = img.cuda()

In [7]:
model.cuda()
output = model(img)

RuntimeError: CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
print(output.shape)

torch.Size([1, 20, 7, 7])
